In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from ImagesloaderRAM import *
from Trainfunction import *
from Savemodel import *
from Mycudatransformation import *
from MynnModule import *
from PSNR import *

import torchvision.transforms as T
import torchvision.utils as tu

import numpy as np


from time import time

dtype = torch.FloatTensor
gpu_dtype=torch.cuda.FloatTensor

In [5]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
    
print('using device:', device)

using device: cuda


# Train and Test the single frame denoiser
## Train

In [ ]:
# Loading the dataset of images


Data_preprocessing=T.Compose([ T.RandomCrop(64),
                                        T.ToTensor()])  
                                        
trainset = ImageFolderRAM('./train',0.1,Data_preprocessing,Randomnoise=False, loader= RGB_loader, loadram='cpu')

trainloader = torch.utils.data.DataLoader(trainset, shuffle=True, batch_size=64,num_workers=0)



### Training from scratch

In [6]:
# Choose the model of neural networks architectures 
Denoiser = SSFD_C().to(device)

# Choose the loss function 
loss = nn.L1Loss().to(device)

# Choose the Optimizer
optimizer = optim.Adam(Denoiser.parameters(),lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.00001, amsgrad=False)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.9, patience=5000, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)



In [ ]:
trainserver(Denoiser,trainloader, loss, optimizer, scheduler, num_epochs = 500000,save_every=25000,loss_every=3,filename='SSFD_G_NB')

In [ ]:
def trainserver(model,loader_train, loss_fn, optimizer,scheduler, num_epochs = 1,save_every=1,loss_every=10,filename='denoiser'):
    
    loss_history=[]
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()
        
        for t, (x, y) in enumerate(loader_train):
            if torch.cuda.is_available():
                
                x_var = Variable(x.type(gpu_dtype))
                y_var = Variable(y.type(gpu_dtype))
            
            else: 
                x_var = Variable(x.type(dtype))
                y_var = Variable(y.type(dtype))

            scores = model(x_var)
            
            loss = loss_fn(scores, y_var)
            
            if (t + 1) % loss_every == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.data))
                loss_history.append(loss)
                

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        if (epoch+1) % save_every == 0:
                Save_modelloss(model,optimizer,loss_history,filename+'%s ' %int(epoch+1))
                
    
        scheduler.step(loss.data)

## Test denoiser and save images